In [1]:
import matplotlib.pyplot as plt 
import scipy.misc
import numpy as np
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation, GlobalAveragePooling2D
from keras.models import Model
from keras.utils.layer_utils import print_summary
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

from keras import __version__ as kv
from scipy import __version__ as sv
from matplotlib import __version__ as mv
from numpy import __version__ as nv

print kv
print sv
print mv
print nv

Using TensorFlow backend.


2.0.5
0.19.1
2.0.0
1.13.0


In [2]:
# Local files
import utils
reload(utils)
import utils2
reload(utils2)

<module 'utils2' from 'utils2.pyc'>

# Data

### Config

In [3]:
trained_model_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/trained_models/4.0-as-FCN/'

In [4]:
train_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/train/50x50/'
valid_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/validate/50x50/'

In [5]:
input_shape = (50, 50, 1)
num_classes = 3

### Convert tif2png

Done in 1.1-mph_as_kd-baseline-simple_CNN_from_scratch.ipynb

### Create Generators

In [6]:
train_datagen = ImageDataGenerator()#rescale=1./255)

test_datagen = ImageDataGenerator()#rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(50, 50),
        batch_size=20,
        class_mode='categorical', 
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        valid_dir, 
        target_size=(50, 50),
        shuffle=False,
        batch_size=10,
        class_mode='categorical',
        color_mode='grayscale')

Found 1996 images belonging to 3 classes.
Found 499 images belonging to 3 classes.


# Experiments

## Baseline FCN (using https://github.com/asmith26/courses/blob/master/deeplearning1/nbs/lesson7.ipynb)

In [7]:
p = 0

classifier_input = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

# # Following based on https://github.com/asmith26/courses/blob/master/deeplearning1/nbs/lesson7.ipynb
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64,3,3, activation='relu', border_mode='same')(x)
x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)
x = Conv2D(64,3,3, activation='relu', border_mode='same')(x)
x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)
x = Conv2D(3,3,3, border_mode='same')(x)
x = Dropout(p)(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax')(x)


model = Model(outputs=x, inputs=classifier_input)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (3, 3), padding="same")`



In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 50, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 12, 64)        256       
__________

In [9]:
lr = 0.001
K.set_value(model.optimizer.lr, lr)

model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=60)

Epoch 1/20
100/100 [==============================] - 17s - loss: 0.4426 - acc: 0.8285 - val_loss: 1.1730 - val_acc: 0.5025
Epoch 2/20
100/100 [==============================] - 17s - loss: 0.2755 - acc: 0.9006 - val_loss: 0.2597 - val_acc: 0.9165
Epoch 3/20
100/100 [==============================] - 17s - loss: 0.2093 - acc: 0.9184 - val_loss: 0.2634 - val_acc: 0.9115
Epoch 4/20
100/100 [==============================] - 17s - loss: 0.1753 - acc: 0.9310 - val_loss: 0.2342 - val_acc: 0.9182
Epoch 5/20
100/100 [==============================] - 17s - loss: 0.1324 - acc: 0.9555 - val_loss: 0.1803 - val_acc: 0.9466
Epoch 6/20
100/100 [==============================] - 17s - loss: 0.1177 - acc: 0.9634 - val_loss: 0.2074 - val_acc: 0.9381
Epoch 7/20
100/100 [==============================] - 17s - loss: 0.1073 - acc: 0.9609 - val_loss: 0.1622 - val_acc: 0.9633
Epoch 8/20
100/100 [==============================] - 17s - loss: 0.0948 - acc: 0.9675 - val_loss: 0.1953 - val_acc: 0.9399
Epoch 9/

In [10]:
lr = 0.0001
K.set_value(model.optimizer.lr, lr)

model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=60)

Epoch 1/5
100/100 [==============================] - 17s - loss: 0.0115 - acc: 0.9950 - val_loss: 0.2141 - val_acc: 0.9666
Epoch 2/5
100/100 [==============================] - 17s - loss: 0.0024 - acc: 0.9995 - val_loss: 0.2229 - val_acc: 0.9616
Epoch 3/5
100/100 [==============================] - 17s - loss: 9.2996e-04 - acc: 1.0000 - val_loss: 0.2063 - val_acc: 0.9683
Epoch 4/5
100/100 [==============================] - 17s - loss: 8.9355e-04 - acc: 1.0000 - val_loss: 0.2363 - val_acc: 0.9649
Epoch 5/5
100/100 [==============================] - 17s - loss: 3.9482e-04 - acc: 1.0000 - val_loss: 0.2480 - val_acc: 0.9616


In [11]:
lr = 0.00001
K.set_value(model.optimizer.lr, lr)

model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=2,
        validation_data=validation_generator,
        validation_steps=60)

Epoch 1/2
100/100 [==============================] - 17s - loss: 2.7897e-04 - acc: 1.0000 - val_loss: 0.1935 - val_acc: 0.9683
Epoch 2/2
100/100 [==============================] - 17s - loss: 2.8006e-04 - acc: 1.0000 - val_loss: 0.2409 - val_acc: 0.9633
